In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import random
import scipy.sparse.csgraph as sps
import copy
import time
import queue as Q

In [3]:
widgetMat = [
            ['A', 'E', 'D', 'C', 'A'],
            ['B', 'E', 'A', 'C', 'D'],
            ['B', 'A', 'B', 'C', 'E'],
            ['D', 'A', 'D', 'B', 'D'],
            ['B', 'E', 'C', 'B', 'D']]

distanceMat = [
            [0, 1064, 673, 1401, 277],
            [1064, 0, 958, 1934, 337],
            [673, 958, 0, 1001, 399],
            [1401, 1934, 1001, 0, 387],
            [277, 337, 399, 387, 0]]

minDistMat, predMat = sps.dijkstra(distanceMat, return_predecessors=True)

distMatDic = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}
revDistMatDic = {v: k for k, v in distMatDic.items()}

def calculateMiles(path):
    miles = 0
    for i in range(len(path)-1):
        miles += distanceMat[distMatDic[path[i]]][distMatDic[path[i+1]]]
    return miles
    
def calculateMinMiles(current, path):
    if not current:
        miles = 0
    else: miles = minDistMat[distMatDic[current]][distMatDic[path[0]]]
    for i in range(len(path)-1):
        miles += minDistMat[distMatDic[path[i]]][distMatDic[path[i+1]]]
    return miles

# def totalMiles(current, remainingWidgets):
#     totalMiles = 0
#     for path in remainingWidgets:
#         if path:
#             totalMiles += calculateMinMiles(current, path)
#     return totalMiles
    
def maxMiles(current, remainingWidgets):
    maxMiles = 0
    for path in remainingWidgets:
        if path:
            maxMiles = max(maxMiles, calculateMinMiles(current, path))
    return maxMiles

# def maxMileDiff(current, remainingWidgets):
#     maxMiles = 0
#     for path in remainingWidgets:
#         if path:
#             maxMiles = max(maxMiles, calculateMileDiff(current, path))
#     return maxMiles

def nonrepeatedOrd(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def shortestPath(cur, des):
    path = []
    temp = copy.deepcopy(cur)
    while temp != des:
        num = predMat[distMatDic[des]][distMatDic[temp]]
        temp = revDistMatDic[num]
        path.append(temp)
    if cur in path: path.pop(0)
    return nonrepeatedOrd(path)

def widgetGenerator(n):
    random.seed(1118)
    widgets = []
    letters = 'ABCDE'
    for i in range(5):
        widget = []
        for j in range(n):
            if j > 0:
                choice = letters.replace(widget[len(widget)-1], "")
                widget.append(random.choice(choice))
            else: widget.append(random.choice(letters))
        widgets.append(widget)
    return widgets

In [ ]:
class widgetState():
    def __init__(self, path, remainingWidget):
        self.remainingWidget = 
        self.path = path
        self.heuristic = 0
        
    def __eq__(self, widget):
        return self.remainingWidget == widget.remainingWidget and self.path == widget.path
#         return self.path == widget.path
#         return self.heuristic == widget.heuristic
    
    def __lt__(self, widget):
        return self.heuristic < widget.heuristic

#Pick Widget

    def popComp(self, component):
        widgets = copy.deepcopy(self.remainingWidget)
        for widg in widgets:
            if widg and widg[0] == component:
                widg.pop(0)
        return widgets
    
#     def calculateMileDiff(self, current, goal, remainingWidgets):
#         print("totalMilesDiff:")
# #         print(totalMiles(current, self.remainingWidget) - totalMiles(goal, self.popComp(goal)))
#         print(maxMiles(current, self.remainingWidget) - maxMiles(goal, self.popComp(goal)))
#         return maxMiles(current, self.remainingWidget) - maxMiles(goal, self.popComp(goal))
    
#     def maxMileDiff(self, current, remainingWidgets):
#         maxMileDiff = -np.inf
#         for path in remainingWidgets:
#             if path:
#                 maxMileDiff = max(maxMileDiff, self.calculateMileDiff(current, path[0], self.remainingWidget))
#         print("maxMileDiff:")
#         print(maxMileDiff)
#         return maxMileDiff
                
#     def belongLongestWidget(self, stop):
#         for widg in self.remainingWidget:
#             if widg and widg[0] == stop and len(widg) == len(max(self.remainingWidget,key=len)):
#                 return True
#         return False
    
    def adjacent_minStops(self):
        adjList = []
        lastStop = None
        frontList = []
        for path in self.remainingWidget:
            if path:
                frontList.append(path[0])
        if self.path:
            lastStop = self.path[-1]
        for node in set(frontList):
            if node != lastStop:
                path = copy.deepcopy(self.path)
                path.append(node)
                pickWidgets = self.popComp(node)
                adjList.append(widgetState(path, pickWidgets))
        return adjList
    
    def adjacent_minMiles(self):
        adjList = []
        lastStop = None
        frontList = []
        for path in self.remainingWidget:
            if path:
                frontList.append(path[0])
        if self.path:
            lastStop = self.path[-1]
#         maxMile = self.maxMileDiff(lastStop, self.remainingWidget)
        for node in set(frontList):
            if node != lastStop:
                path = copy.deepcopy(self.path)
                if lastStop:
                    for stop in shortestPath(lastStop, node):
                        path.append(stop)
                else: path.append(node)
                pickWidgets = self.popComp(node)
                adjList.append(widgetState(path, pickWidgets))
        return adjList
        
    def printState(self):
        print("widgets:")
        print(self.remainingWidget)
        print("path:")
        print(self.path)
        print("heuristic:")
        print(self.heuristic)